In [1]:
def compute_aligned_gradients(conn_matrices, mean_grad_for_alignment, data_reduction_algorithm = 'dm'):
    
    '''
    
    Function that computes the alligned gradients from all subject connectivity matrices
    
    Input:
    - variable containing all subject connectivity matrices
    - array of the gradients to which the individual gradients should be aligned (the mean connectivity gradients)
    - data_reduction_algorithm: used to compute the gradients. Options: 'dm' (diffusion map embedding; default), 'le' (laplacian eigenmaps), 'pca' (principal component analysis)
    
    Output (dictionary containing arrays):  
    - array_aligned_gradients
    - array_aligned_G1
    - array_aligned_G2
    - array_aligned_G3
    
    Gradients computed from the mean connectivity matrix with default parameters (diffusion map embedding, 10 gradients, threshold (fit -> sparsity = 0.90)
    
    '''
    
    list_aligned_gradients = []  # will contain all participants (1014), all parcels (400), all gradients (10)
    list_aligned_G1 = []  # will contain all participants (1014), all parcels (400) -> values = loadings for GRADIENT 1
    list_aligned_G2 = []  # will contain all participants (1014), all parcels (400) -> values = loadings for GRADIENT 2
    list_aligned_G3 = []  # will contain all participants (1014), all parcels (400) -> values = loadings for GRADIENT 3

    # loop over all the connectivity matrices
    for i in range(len(conn_matrices)):
        # setting model parameters for gradients to be computed across subjects - with procrustes alignment
        #grad_procr = GradientMaps(n_components=10, random_state=0, approach = data_reduction_algorithm, alignment='procrustes', kernel='normalized_angle')  # specify alignment method here (procrustes)
        grad_procr = GradientMaps(n_components=10, random_state=0, approach = data_reduction_algorithm, alignment='procrustes')  # specify alignment method here (procrustes)

        # computing
        # note that by using an alignment method, .fit yields a variable (grad_procr) containing both types of gradients, callable with: .gradients_ (original) and .aligned_ 
          # use ._gradients for mean_grad_for_alignment (mean grad was not even calculated with a reference so doesn't have ._aligned) and use .aligned_ for grad_procr 

        grad_procr.fit(conn_matrices[i], reference=mean_grad_for_alignment)  # align to the gradients of the gradients produced by the mean matrix (reference) 

        # append array to lists results (.T is necessary in order to be able to first access the gradients layer (10) so that can index the desired gradient, which will then contain all the parcels (400)
        list_aligned_gradients.append(grad_procr.aligned_)
        list_aligned_G1.append(grad_procr.aligned_.T[0])
        list_aligned_G2.append(grad_procr.aligned_.T[1])
        list_aligned_G3.append(grad_procr.aligned_.T[2])

    # make gradient lists into arrays (for analyses)    
    array_aligned_gradients = np.array(list_aligned_gradients)
    array_aligned_G1 = np.array(list_aligned_G1)
    array_aligned_G2 = np.array(list_aligned_G2)
    array_aligned_G3 = np.array(list_aligned_G3)
        
        
    ### dictionary to output
    
    dict_output = {'array_aligned_gradients': array_aligned_gradients, 'array_aligned_G1': array_aligned_G1, 'array_aligned_G2': array_aligned_G2, 'array_aligned_G3': array_aligned_G3}
    
    
    return dict_output

In [2]:
def SpinPermutationTest_Schaefer400(x, y, correlation_type = 'pearson', already_in_fs5_space = False):
    
    '''
    
    Function that conducts spin permutation testing (for pearson correlation) specifically for data in Schaefer400 parcellation (len = 400) or already in fs5 space
    using enigmatoolbox.permutation_testing package
    
    Input:
    - x: data to correlate 
    - y: data to correlate 
    - correlation_type: 'pearson' or 'spearman'
    - already_in_fs5_space: if True, data is already in fs5 space. If False, data is in Schaefer 400 and will be converted within this function to fs5
    
    Output (display):  
    - spin permutation p-value
    - plotted null distribution of generated correlations

   
    '''
    
    from enigmatoolbox.permutation_testing import spin_test, shuf_test
    
    if already_in_fs5_space:
        sample_1_fs5 = x
        sample_2_fs5 = y
    
    
    else:
    
        ### Project x and y data (from Schaefer 400 parcellation) to fsaverage5's 20484 vertices (required for enigmatoolbox.permutation_testing package)

        sample_1_fs5 = []
        sample_2_fs5 = []

        # iterate over the 20484 vertices in fsaverage5
        for i in range(len(schaefer_400_fs5)):

            if schaefer_400_fs5[i] == 0:  # corresponds to the midline
                # append to the lists of fs5_tvals: 0
                sample_1_fs5.append(0)
                sample_2_fs5.append(0)

            else:
                # append to the lists of fs5_tvals: the unimodal-heteromodal gradient eigenvalue of the corresponding Schaefer parcel (here parcel value [i] - 1 because parcel numbers go from 1-400 instead of 0-399 as required for indexing)
                sample_1_fs5.append(x[schaefer_400_fs5[i]-1])
                sample_2_fs5.append(y[schaefer_400_fs5[i]-1])

        # change the zeros into nan (couldn't nan directly because then it made the array content strings
        sample_1_fs5[sample_1_fs5 == 0] = np.nan
        sample_2_fs5[sample_2_fs5 == 0] = np.nan

        # transform list into array
        sample_1_fs5 = np.asarray(sample_1_fs5)
        sample_2_fs5 = np.asarray(sample_2_fs5)
    

    
    ### Spin permutation testing 
    
    # spin permutation testing for two cortical maps (output of spin_test is the p-value and the null distribution)
    spin_test_p, spin_test_d = spin_test(sample_1_fs5, sample_2_fs5, surface_name='fsa5', parcellation_name='aparc', type=correlation_type, n_rot=1000, null_dist=True)
    
    
    # print spin permutation test
    print(f"Spin permutation test p-value: {spin_test_p}")
    
    
    
    ### Plot null distribution of generated correlations
    
    # To better interpret statistical significance, we can plot the null distribution of generated correlations (i.e., “spun” or “shuffled” correlations) and overlay the correlation coefficient obtained from the empirical (i.e., real) brain maps.
    
    fig, ax = plt.subplots(1, figsize=(15, 3))

    ax.hist(spin_test_d, bins=50, density=True, color="blue", edgecolor='white', lw=0.5)
    ax.set_xlabel('Null correlations')
    ax.set_ylabel('Density')
    ax.set_title('Null distribution of generated correlations')
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)


    fig.tight_layout()
    plt.show()
    
    
    # return the spin test p value and null distribution of r values in a dictionary format
    return {'spin_test_p': spin_test_p, 'spin_test_d': spin_test_d}


In [6]:
def variability_parcel_std(subject_grad_M, subject_grad_F, sample_modality, subject_grad=None, plot_in_fsa5 = False, save_plots = False, output_dir = None):
    
    '''
    
    Function that computes variability at the parcel level: Visualizing standard deviation by parcels in both sexes separately in order to visualize which sex and parcels have most variability
    - computing the 'difference score' of std (male std - female std) as quantification
    - testing the significance of the sex difference using Levene's test for homogeneity of variance (+ FDR-correction)
    - computing a network breakdown of significant differences (pie charts)
    
    Input variables:
    - subject_grad: aligned gradient loadings of all subjects - shape array: number of subjects x number of parcels (optional)
    - subject_grad_M: aligned gradient loadings of male subjects - shape array: number of subjects x number of parcels
    - subject_grad_F: aligned gradient loadings of female subjects - shape array: number of subjects x number of parcels
    - plot_in_fsa5: if the results should be plotted in fsa5 - default False
    - save_plots: True/False if want to save screenshots of plotted hemispheres and network breakdown of significant differences
    - output_dir: path to output directory (e.g. resdir_fig, i.e. '/data/p_02667/sex_diff_gradients/results/figures/')
    - sample_modality: will be included in label of saved plot names (e.g., 'HCP_fc_G1')
    
    Output:
    - Number of parcels for which there males have a statistically significant larger variance than females; and for which females have a statistically significant larger variance than males (before and after FDR correction)
    - plotted hemispheres: *STD across all subjects, STD males, STD females, difference STD M - F, p-values Levene's test, q-values Levene's test (FDR), difference STD for p sig, *difference STD for q sig
    - nested pie chart: breakdown of statistically significant sex differences by network
    - saves in output_dir: *STD_plotted_hemispheres_across_sexes, *STD_plotted_hemispheres_sex_differences_fdr_corr, and STD_pie_chart_sex_diff_netw
    - returns array with differences in STD
    
    '''
    
    ### Compute std per parcel, across all subjects, and for males and females separately
    if subject_grad is not None:
        std_grad = np.std(subject_grad, axis=0)
    std_grad_M = np.std(subject_grad_M, axis=0)
    std_grad_F = np.std(subject_grad_F, axis=0)


    ### Compute the difference between males and female std (where positive scores show greater female variability)
    std_grad_sexdiff = std_grad_F - std_grad_M

    
    ### Significance testing of the differences

    # list that will contain the p values of the Levene's test for homogeneity of variance (per parcel) -> p < 0.05 mean NOT homogeneous variance, meaning that we can interpret the variability (as provided by difference of STD) as statistically significant
    p_val_levene_grad_male_vs_female = []

    # loop over 400 parcels
    for i in range(len(subject_grad_M.T)):

        # test for homogeneity of variance within this parcel (between males and females) - [1] indexes the p-value -> append p-value to list
        p_val_levene_grad_male_vs_female.append(stats.levene(subject_grad_M.T[i], subject_grad_F.T[i])[1])

    p_val_levene_grad_male_vs_female = np.array(p_val_levene_grad_male_vs_female)


    # compute the FDR-corrected q values of G1 sex differences in variance as given from Levene's test pvalues
    fdr_corr_p_val_levene_grad_male_vs_female = fdrcorrection(p_val_levene_grad_male_vs_female)[1]



    # list that will contain difference score (F - M) of STD only for parcels who show statistical inhomogeneity of variance (i.e., Levene's test: p < 0.05) - NOT FDR corrected (just to see patterns)
    std_grad_levene_sig_sexdiff = []

    for i in range(400):
        if p_val_levene_grad_male_vs_female[i] <= 0.05:
            std_grad_levene_sig_sexdiff.append(std_grad_sexdiff[i])
        else:
            std_grad_levene_sig_sexdiff.append(float('nan'))

    std_grad_levene_sig_sexdiff = np.array(std_grad_levene_sig_sexdiff)


    print(f"Number of parcels for which there males have a statistically significant larger variance than females: {np.sum(np.array(std_grad_levene_sig_sexdiff) < 0, axis=0)}")
    print(f"Number of parcels for which there females have a statistically significant larger variance than males: {np.sum(np.array(std_grad_levene_sig_sexdiff) > 0, axis=0)}")



    # list that will contain difference score (M - F) of STD only for parcels who show statistical inhomogeneity of variance (i.e., Levene's test: p < 0.05) AFTER FDR correction (so where q < 0.05)
    std_grad_fdr_corr_levene_sig_sexdiff = []
    

    for i in range(400):
        if fdr_corr_p_val_levene_grad_male_vs_female[i] <= 0.05:
            std_grad_fdr_corr_levene_sig_sexdiff.append(std_grad_sexdiff[i])
            
        else:
            std_grad_fdr_corr_levene_sig_sexdiff.append(float('nan'))
            

    std_grad_fdr_corr_levene_sig_sexdiff = np.array(std_grad_fdr_corr_levene_sig_sexdiff)
    
    
    # for plotting (colormap range determination) identify if there are only 
    
    # if there are no parcels for which std is male > female (i.e. only + values) -> use max as anchor
    if np.sum(np.array(std_grad_fdr_corr_levene_sig_sexdiff) < 0, axis=0) == 0:
        color_range_custom = (np.nanmax(std_grad_fdr_corr_levene_sig_sexdiff)*-1, np.nanmax(std_grad_fdr_corr_levene_sig_sexdiff))
        
    # if there are no parcels for which std is male < female (i.e. only - values) -> use min as anchor
    elif np.sum(np.array(std_grad_fdr_corr_levene_sig_sexdiff) > 0, axis=0) == 0:
        color_range_custom = (np.nanmin(std_grad_fdr_corr_levene_sig_sexdiff), np.nanmin(std_grad_fdr_corr_levene_sig_sexdiff)*-1)   
    
    # if there are both parcels with std male > female and female > male, make colormap symetrical
    else:
        color_range_custom = 'sym'    
        

    print(f"Number of parcels for which there males have a statistically significant larger variance than females after FDR-correction: {np.sum(np.array(std_grad_fdr_corr_levene_sig_sexdiff) < 0, axis=0)}")
    print(f"Number of parcels for which there females have a statistically significant larger variance than males after FDR-correction: {np.sum(np.array(std_grad_fdr_corr_levene_sig_sexdiff) > 0, axis=0)}")
    
    



    ### Find min and max std across sexes (this is for plotting the color bar in the plotted hemispheres)

    if min(std_grad_M) < min(std_grad_F):
        min_std = min(std_grad_M)
    else:
        min_std = min(std_grad_F)

    print(f"\nMinimum SD: Males = {round(min(std_grad_M), 3)}; Females = {round(min(std_grad_F), 3)}")


    if max(std_grad_M) > max(std_grad_F):
        max_std = max(std_grad_M)
    else:
        max_std = max(std_grad_F)

    print(f"Maximum SD: Males = {round(max(std_grad_M), 3)}; Females = {round(max(std_grad_F), 3)}")
    
    
    
    
    
    
    ### plot the standard deviations on hemispheres
    
      
    
    
    ### IF plotting in fsa5
    
    if plot_in_fsa5:
        
        # transform all that there is to plot into fsa5
        if subject_grad is not None:
            std_to_labels = schaefer400_to_fs5(schaefer400_data = std_grad, seven_networks=False) 
        std_to_labels_M = schaefer400_to_fs5(schaefer400_data = std_grad_M, seven_networks=False) 
        std_to_labels_F = schaefer400_to_fs5(schaefer400_data = std_grad_F, seven_networks=False) 
        std_grad_sexdiff_to_labels = schaefer400_to_fs5(schaefer400_data = std_grad_sexdiff, seven_networks=False) 
        levene_pval_to_labels = schaefer400_to_fs5(schaefer400_data = p_val_levene_grad_male_vs_female, seven_networks=False) 
        fdr_corr_levene_pval_to_labels = schaefer400_to_fs5(schaefer400_data = fdr_corr_p_val_levene_grad_male_vs_female, seven_networks=False) 
        std_grad_sig_Levene_sexdiff_to_labels = schaefer400_to_fs5(schaefer400_data = std_grad_levene_sig_sexdiff, seven_networks=False) 
        std_grad_sig_fdr_corr_levene_sexdiff_to_labels = schaefer400_to_fs5(schaefer400_data = std_grad_fdr_corr_levene_sig_sexdiff, seven_networks=False) 


        # need to plot right and left hemispheres with fsaverage mesh (instead of loading conte69) because I transformed the data in fsaverage5 (already in format: 20484 vertices) unlike conte69 (64984 vertices)
        mesh_paths = {
           'L': nilearn.datasets.fetch_surf_fsaverage('fsaverage5')['pial_left'],
           'R': nilearn.datasets.fetch_surf_fsaverage('fsaverage5')['pial_right']
        }
        surf_lh = brainspace.mesh.mesh_io.read_surface(mesh_paths['L'])
        surf_rh = brainspace.mesh.mesh_io.read_surface(mesh_paths['R'])

    
    else:
        
        # defining labeling scheme and mask for normal Schaefer 400 parcellation
        labeling = load_parcellation('schaefer', scale=400, join=True)
        surf_lh, surf_rh = load_conte69()

        mask = labeling != 0
        
        
        # mapping to labels
        if subject_grad is not None:
            std_to_labels = map_to_labels(std_grad, labeling, mask=mask, fill=np.nan)
        std_to_labels_M = map_to_labels(std_grad_M, labeling, mask=mask, fill=np.nan)  
        std_to_labels_F = map_to_labels(std_grad_F, labeling, mask=mask, fill=np.nan)  
        std_grad_sexdiff_to_labels = map_to_labels(std_grad_sexdiff, labeling, mask=mask, fill=np.nan)  
        levene_pval_to_labels = map_to_labels(p_val_levene_grad_male_vs_female, labeling, mask=mask, fill=np.nan)  
        fdr_corr_levene_pval_to_labels = map_to_labels(fdr_corr_p_val_levene_grad_male_vs_female, labeling, mask=mask, fill=np.nan) 
        std_grad_sig_Levene_sexdiff_to_labels = map_to_labels(std_grad_levene_sig_sexdiff, labeling, mask=mask, fill=np.nan)  
        std_grad_sig_fdr_corr_levene_sexdiff_to_labels = map_to_labels(std_grad_fdr_corr_levene_sig_sexdiff, labeling, mask=mask, fill=np.nan) 
        
        
    

    # will contain the different plots
    handles = []

    
    if subject_grad is not None:
        plotted_hemispheres_std = plot_hemispheres(surf_lh, 
                                                 surf_rh, 
                                                 array_name=std_to_labels, 
                                                 embed_nb = True, 
                                                 size=(1200, 200), 
                                                 color_range = (min_std, max_std),
                                                 cmap='YlGn', 
                                                 color_bar=True, 
                                                 nan_color = (0.7, 0.7, 0.7, 1),
                                                 label_text=['STD'], 
                                                 zoom=1.55,
                                                 screenshot = save_plots,
                                                 filename = output_dir+sample_modality+'_STD_plotted_hemispheres_across_sexes.png')

        handles.append(plotted_hemispheres_std)



    

    plotted_hemispheres_M_std = plot_hemispheres(surf_lh, 
                                                 surf_rh, 
                                                 array_name=std_to_labels_M, 
                                                 embed_nb = True, 
                                                 size=(1200, 200), 
                                                 color_range = (min_std, max_std),
                                                 cmap='YlGn', 
                                                 color_bar=True, 
                                                 nan_color = (0.7, 0.7, 0.7, 1),
                                                 label_text=['Males'], 
                                                 zoom=1.55,
                                                 screenshot = save_plots,
                                                 filename = output_dir+sample_modality+'_STD_plotted_hemispheres_Male.png')

    handles.append(plotted_hemispheres_M_std)



   

    plotted_hemispheres_F_std = plot_hemispheres(surf_lh, 
                                                 surf_rh, 
                                                 array_name=std_to_labels_F, 
                                                 embed_nb = True, 
                                                 size=(1200, 200), 
                                                 color_range = (min_std, max_std),
                                                 cmap='YlGn', 
                                                 color_bar=True, 
                                                 nan_color = (0.7, 0.7, 0.7, 1),
                                                 label_text=['Females'], 
                                                 zoom=1.55,
                                                 screenshot = save_plots,
                                                 filename = output_dir+sample_modality+'_STD_plotted_hemispheres_Female.png')

    handles.append(plotted_hemispheres_F_std)



    

    plotted_hemispheres_std_grad_sexdiff = plot_hemispheres(surf_lh, 
                                                 surf_rh, 
                                                 array_name=std_grad_sexdiff_to_labels, 
                                                 embed_nb = True, 
                                                 size=(1200, 200), 
                                                 cmap='bwr', 
                                                 color_bar=True,
                                                 color_range='sym',
                                                 nan_color = (0.7, 0.7, 0.7, 1),
                                                 label_text=['STD(F) - STD (M)'], 
                                                 zoom=1.55,
                                                 screenshot = save_plots,
                                                 filename = output_dir+sample_modality+'_STD_plotted_hemispheres_stdF_minus_stdM.png')           
    

    handles.append(plotted_hemispheres_std_grad_sexdiff)



    

    plotted_hemispheres_levene_pval = plot_hemispheres(surf_lh, 
                                                 surf_rh, 
                                                 array_name=levene_pval_to_labels, 
                                                 embed_nb = True, 
                                                 size=(1200, 200), 
                                                 cmap='plasma_r', 
                                                 color_bar=True, 
                                                 nan_color = (0.7, 0.7, 0.7, 1),
                                                 label_text=["Levene's p-vals"], 
                                                 zoom=1.55,
                                                 screenshot = False
                                                )

    handles.append(plotted_hemispheres_levene_pval)



     

    plotted_hemispheres_fdr_corr_levene_pval = plot_hemispheres(surf_lh, 
                                                 surf_rh, 
                                                 array_name=fdr_corr_levene_pval_to_labels, 
                                                 embed_nb = True, 
                                                 size=(1200, 200), 
                                                 cmap='plasma_r', 
                                                 color_bar=True, 
                                                 nan_color = (0.7, 0.7, 0.7, 1),
                                                 label_text=["FDR-corrected\nLevene's q-vals"], 
                                                 zoom=1.55,
                                                 screenshot = False
                                                )

    handles.append(plotted_hemispheres_fdr_corr_levene_pval)



    

    plotted_hemispheres_std_grad_levene_sig_sexdiff = plot_hemispheres(surf_lh, 
                                                 surf_rh, 
                                                 array_name=std_grad_sig_Levene_sexdiff_to_labels, 
                                                 embed_nb = True, 
                                                 size=(1200, 200), 
                                                 cmap='bwr', 
                                                 color_bar=True, 
                                                 color_range='sym',
                                                 nan_color = (0.7, 0.7, 0.7, 1),
                                                 label_text=['Differences STD\nfor Levene p<.05'], 
                                                 zoom=1.55,
                                                 screenshot = False
                                                )

    handles.append(plotted_hemispheres_std_grad_levene_sig_sexdiff)


     

    plotted_hemispheres_std_grad_fdr_corr_levene_sig_sexdiff = plot_hemispheres(surf_lh, 
                                                 surf_rh, 
                                                 array_name=std_grad_sig_fdr_corr_levene_sexdiff_to_labels, 
                                                 embed_nb = True, 
                                                 size=(1200, 200), 
                                                 cmap='bwr', 
                                                 color_bar=True, 
                                                 color_range=color_range_custom,
                                                 nan_color = (0.7, 0.7, 0.7, 1),
                                                 label_text=['Differences STD\nfor FDR-corrected\nLevene q<.05'], 
                                                 zoom=1.55,
                                                 screenshot = save_plots,
                                                 filename = output_dir+sample_modality+'_STD_plotted_hemispheres_sex_differences_fdr_corr.png')

    handles.append(plotted_hemispheres_std_grad_fdr_corr_levene_sig_sexdiff)


    display(*handles)


    

    ### results breakdown by network
    
    # take the yeo7 network array labels corresponding to the parcellation in use (yeo7 or yeo17 Schaefer 400)
    if plot_in_fsa5:
        yeo7_networks_labels = yeo_17_2_7_networks_array_labels
        
    else:
        yeo7_networks_labels = yeo7_networks_array_labels


    ## written breakdown

    # counting number of significant parcels
    # storing the Q values in a list (where non significant Q values are marked as 1 -> for later potential scatterplot visualization)
    # making a dictionary that counts the number of significant parcels per yeo network
    # making dictionaries that count the number of significant parcels per yeo network by sex

    sig_Q_vals = []
    count_sig = 0
    count_sig_M = 0
    count_sig_F = 0
    count_sig_per_network = {"visual": 0, "sensory motor": 0, "DMN": 0, "dorsal attention": 0, "ventral attention": 0, "limbic": 0, "fronto parietal": 0}
    count_sig_per_network_bysex = {"visual": [0, 0], "sensory motor": [0,0], "DMN": [0,0], "dorsal attention": [0,0], "ventral attention": [0,0], "limbic": [0,0], "fronto parietal": [0,0]} # M: [0], F: [1]

    # loop over 400 parcels
    for i in range(len(fdr_corr_p_val_levene_grad_male_vs_female)):

        if fdr_corr_p_val_levene_grad_male_vs_female[i] < 0.05:
            count_sig += 1
            count_sig_per_network[yeo7_networks_labels[i]] += 1
            sig_Q_vals.append(1)

            # positive t-values mean male < female: increment the first item of the list fort given label
            if std_grad_sexdiff[i] < 0:
                count_sig_M += 1
                count_sig_per_network_bysex[yeo7_networks_labels[i]][0] += 1

            # positive t-values mean female > male: increment the second item of the list for the given label
            else:
                count_sig_F += 1
                count_sig_per_network_bysex[yeo7_networks_labels[i]][1] += 1

        else:
            sig_Q_vals.append(0)

    print(f"Number of significant parcels: {count_sig}\n")
    print(f"Number of significant parcels for males: {count_sig_M}")
    print(f"Number of significant parcels for females: {count_sig_F}\n")
    print("Number of significant parcels in each Yeo network (across sexes):")

    # using ANSI escape sequences to underline -> bold: \033[1m ; underline: \033[4m ; end: \033[0m
    for i in range(len(count_sig_per_network)):
        print(f"- {list(count_sig_per_network.keys())[i]}: \033[4m{count_sig_per_network[list(count_sig_per_network.keys())[i]]}\033[0m out of {yeo7_networks_labels.tolist().count(network_names[i])} ({round(count_sig_per_network[list(count_sig_per_network.keys())[i]] / yeo7_networks_labels.tolist().count(network_names[i]) * 100, 2)}%) -> \033[1m{round(count_sig_per_network[list(count_sig_per_network.keys())[i]]*100/count_sig,2)}%\033[0m of overall significance")

    print("\n\n")




    ### Nested pie chart

    ## make data plottable
    list_count_sig_per_network_bysex = []

    for label in count_sig_per_network_bysex:
        list_count_sig_per_network_bysex.append(count_sig_per_network_bysex[label])

    vals = np.array(list_count_sig_per_network_bysex)

    outer_colors = ["darkorchid",  # visual
                    "steelblue",  # sensorimotor
                    "indianred",  # dmn
                    "forestgreen",  # dorsal attention
                    "orchid",  # ventral attention
                    "lemonchiffon",  # limbic
                    "orange"]  # frontoparietal
    inner_colors = ['lightblue', 'lightcoral',  # visual
                    'lightblue', 'lightcoral',  # sensorimotor
                    'lightblue', 'lightcoral',  # dmn
                    'lightblue', 'lightcoral',  # dorsal attention
                    'lightblue', 'lightcoral',  # ventral attention
                    'lightblue', 'lightcoral',  # limbic
                    'lightblue', 'lightcoral']  #frontoparietal

    fig, ax = plt.subplots(figsize=(15, 10))

    size = 0.3

    ## plot outer pie
    ax.pie(vals.sum(axis=1), radius=1, labels=count_sig_per_network_bysex.keys(), colors=outer_colors, autopct='%.0f%%', pctdistance=0.85,
           wedgeprops=dict(width=size, edgecolor='white'),  textprops={'fontsize': 20})

    ## plot inner pie

    # make a list (in order) containing the labels (sex - hardcoded) only for sections that have more than 1 count (otherwise label is placeholder: blank)

    labels_only_show_non_null = []

    for network in list_count_sig_per_network_bysex:

        # males
        if network[0] > 0:
            labels_only_show_non_null.append('M')
        else:
            labels_only_show_non_null.append('')

        # females
        if network[1] > 0:
            labels_only_show_non_null.append('F')
        else:
            labels_only_show_non_null.append('')

    ax.pie(vals.flatten(), radius=1-size, labels=labels_only_show_non_null, colors=inner_colors,
           wedgeprops=dict(width=size, edgecolor='white'),  textprops={'fontsize': 15},  labeldistance=0.78)

    ax.set(aspect="equal")
    ax.set_title(f'Breakdown of parcels by network showing a statistically significant sex difference in gradient loadings, by sex', y=1.03, fontsize=20)

    plt.show()

    print('Number of significant parcels by sex:')
    for network in count_sig_per_network_bysex:
        print(f"{network} - Male: {count_sig_per_network_bysex[network][0]}, Female: {count_sig_per_network_bysex[network][1]}")

    print("\n\n")



    ### plot outer and inner pie (without labels)

    fig, disp = plt.subplots(figsize=(15, 10))
    size = 0.3

    disp.pie(vals.sum(axis=1), radius=1, colors=outer_colors, pctdistance=0.85,
           wedgeprops=dict(width=size, edgecolor='black'),  textprops={'fontsize': 20})

    disp.pie(vals.flatten(), radius=1-size, colors=inner_colors,
           wedgeprops=dict(width=size, edgecolor='blacK'),  textprops={'fontsize': 15},  labeldistance=0.78)

    disp.set(aspect="equal")

    plt.show()

    if save_plots:
        ## save figure in directory 
        fig.savefig(output_dir+sample_modality+'_STD_pie_chart_sex_diff_netw.png', dpi=300)
        
        
        
    return std_grad_sexdiff

In [3]:
def plot_reg_results_R(reg_res, contrast_type, save_screenshot = False, output_dir = None, sample_modality = None, categorical_contrast = True):
    
    '''
    Function that plots regression results coming from R script (t-values, p-values, FDR corrected q-values, and t-values corresponding to sig FDR corrected q values)
    
    Input: 
    - reg_res: slm results dataframe containing vales with the following names: t_val, p_val, q_val 
    - contrast type: string indicating the contrast that is being studied, e.g., 'sex' (for figure name and plot titles)
    - save_screenshot: True or False - if you want to save screenshot in output_dir. If True, the plots do not get displayed - so if want to save, need to run with True and then False to leave the plots visible in notebook
    - output_dir: path to output directory (e.g. resdir_fig, i.e. '/data/p_02667/sex_diff_gradients/results/figures/')
    - sample_modality: string, e.g. GSP_local_ct or HCP_fc_G1 (for name of figures saved in output_dir)
    - categorical_contrast: whether the contrast that is being plotted is categorical (e.g., sex) as opposed to continuous (e.g., geodesic distance). Default is True 
    
    Specific to plotting on surf_lh, surf_rh from conte69; for data coming from Schaefer 400 parcellation
    
    '''
    
    # defining the colormaps for plotting on hemispheres of t-values depending on whether the contrast we're looking at is categorical (e.g., male vs female) or continuous (e.g., geodesic distance)
    if categorical_contrast:
        cmap_tval = "bwr_r"  # bwr, _r stands for reversed; using it to match male-blue female-red 
    else:
        cmap_tval = "PRGn"
    
    
    
    # defining 
    
    t_val = reg_res.iloc[:, 0]
    p_val = reg_res.iloc[:, 1]
    q_val = reg_res.iloc[:, 2]
    
    
    # defining labeling scheme and mask
    labeling = load_parcellation('schaefer', scale=400, join=True)
    surf_lh, surf_rh = load_conte69()
    mask = labeling != 0
    
    
    # will contain the different plots
    handles = []
    
    
    ### t-values
    tvals_mapped_to_labels = map_to_labels(np.asarray(reg_res.t_val), labeling, mask=mask, fill=np.nan)  # t[0] because there is a double bracket for the t-values array, need [0] to access the values themselves
    
    tvals_plotted_hemispheres = plot_hemispheres(
        surf_lh, 
        surf_rh, 
        array_name = tvals_mapped_to_labels, 
        embed_nb = True, 
        size = (1400,200), 
        cmap = cmap_tval,  
        color_bar = True, 
        color_range='sym',
        nan_color = (0.7, 0.7, 0.7, 1),
        label_text = ["t-values"],
        zoom = 1.45, 
        screenshot = save_screenshot,
        filename = output_dir+sample_modality+'_plotted_hemispheres_'+contrast_type+'_contrast_t_val.png')
    
    # plot
    handles.append(tvals_plotted_hemispheres)
       
        
    
    ### p-values (uncorrected)
    
    #assigning to new variable using copy() so that changes made in copy will not affect the original array
    pvals = np.asarray(reg_res.p_val.copy())

    # only keep Q-values that are significant (replacing values > 0.05 with nan)
    np.place(pvals, pvals > 0.05, np.nan) 
    
    # this maps shape (400,) turning it inot shape (64984,)
    pvals_mapped_to_labels = map_to_labels(pvals, labeling, mask=mask, fill=np.nan)
    
    # plot
    pvals_plotted_hemispheres =  plot_hemispheres(
        surf_lh, 
        surf_rh, 
        array_name = pvals_mapped_to_labels, 
        embed_nb = True, 
        size = (1400,200), 
        cmap = "plasma_r", 
        color_bar = True, 
        #color_range = color_range_Q,
        nan_color = (0.7, 0.7, 0.7, 1),
        label_text = ["p-values (uncorr.)"],
        zoom = 1.45,
        screenshot = save_screenshot,
        filename = output_dir+sample_modality+'_plotted_hemispheres_'+contrast_type+'_contrast_p_val.png')
    
    handles.append(pvals_plotted_hemispheres)    
        
        
                   
    ### Q-values
    
    #assigning to new variable using copy() so that changes made in copy will not affect the original array
    Qvals = np.asarray(reg_res.q_val.copy())

    # only keep Q-values that are significant (replacing values > 0.05 with nan)
    np.place(Qvals, Qvals > 0.05, np.nan) 
    
    # this maps shape (400,) turning it inot shape (64984,)
    Qvals_mapped_to_labels = map_to_labels(Qvals, labeling, mask=mask, fill=np.nan)
    
    # plot
    Qvals_plotted_hemispheres =  plot_hemispheres(
        surf_lh, 
        surf_rh, 
        array_name = Qvals_mapped_to_labels, 
        embed_nb = True, 
        size = (1400,200), 
        cmap = "plasma_r", 
        color_bar = True, 
        #color_range = color_range_Q,
        nan_color = (0.7, 0.7, 0.7, 1),
        label_text = ["Q-values"],
        zoom = 1.45,
        screenshot = save_screenshot,
        filename = output_dir+sample_modality+'_plotted_hemispheres_'+contrast_type+'_contrast_q_val.png')
    
    handles.append(Qvals_plotted_hemispheres)
    
    
    ### t-values (only FDR-corrected) showing sex differences
    
    ## find t-values
    fdr_corrected_tvals = []
    
    for i in range(len(reg_res.q_val)):
        if reg_res.q_val[i] <= 0.05:
            fdr_corrected_tvals.append(reg_res.t_val[i])
        else:
            fdr_corrected_tvals.append(float('nan'))
    
    fdr_corr_tvals_mapped_to_labels = map_to_labels(np.asarray(fdr_corrected_tvals), labeling, mask=mask, fill=np.nan)
    
    fdr_corr_tvals_plotted_hemispheres = plot_hemispheres(
        surf_lh, 
        surf_rh, 
        array_name = fdr_corr_tvals_mapped_to_labels, 
        embed_nb = True, 
        size = (1400,200), 
        cmap = cmap_tval,
        color_bar = True, 
        color_range='sym',
        nan_color = (0.7, 0.7, 0.7, 1),
        label_text = ["t-values"],
        zoom = 1.45,
        screenshot = save_screenshot,
        filename = output_dir+sample_modality+'_plotted_hemispheres_'+contrast_type+'_contrast_t_val_fdr_corr.png')
    
    # plot
    handles.append('t-values for FDR-corrected q < 0.05: (male: blue, female: red)')  # title
    handles.append(fdr_corr_tvals_plotted_hemispheres)
    
                                           
    return handles

In [1]:
def schaefer400_to_fs5(schaefer400_data, seven_networks=True):
    
    '''
    
    Function that transforms data in Schaefer 400 (7 or 17 networks) space to fsaverage5 space
    
    Input:
    - data in Schaefer 400 space
    - seven_networks (True (7 networks) or False (17 networks)
    
    Output:
    - data in fsaverage5 space
    
    
    '''
    
    # fetch the fsaverage parcellation labeling each of fsaverage5's 20484 vertices with its corresponding Schaefer parcel - seven_networks=False means 17 yeo networks
    from brainstat.datasets import fetch_parcellation
    
    schaefer_400_fs5 = fetch_parcellation("fsaverage5", "schaefer", 400, seven_networks=seven_networks)
    
    
    # list that will contain the fs5 data
    fs5_data = []
    

    # iterate over the 20484 vertices in fsaverage5
    for i in range(len(schaefer_400_fs5)):

        if schaefer_400_fs5[i] == 0:  # corresponds to the midline
            # append to the lists of fs5_tvals: 0
            fs5_data.append(0)

        else:
            # append to the lists of fs5_tvals: the value of the corresponding Schaefer parcel (here parcel value [i] - 1 because parcel numbers go from 1-400 instead of 0-399 as required for indexing)
            fs5_data.append(schaefer400_data[schaefer_400_fs5[i]-1])


    # transform list into array
    fs5_data = np.asarray(fs5_data)

    # change the zeros into nan (couldn't nan directly because then it made the array content strings)
    fs5_data[fs5_data == 0] = np.nan


    return fs5_data

In [4]:
def BN_dispersion_heatmap_matrix(BN_dispersion_contrast_res, df_column_index_with_tvals, df_column_index_with_sigs, colormap_vmin, colormap_vmax, contrast_type, save_plot = False, output_dir = None):
    
    '''
    
    Function that plots a heatmap matrix for the pairwise comparison results of effects on BN dispersion 
    
    Input variables:
    - BN_dispersion_contrast_res: pd dataframe IN CORRECT ORDER (i.e., yielded by my code) containing pairwise comparisons of all possible combinations of 7 Yeo networks and t-values (dataframe also contains p-val, spin p-val and Bonferroni sig (but it is not used here)
    - df_column_index_with_tvals: the number of the index of the column of the dataframe containing the tvalues of interest
    - df_column_index_with_sig: the number of the index of the column of the dataframe containing the significance judgement for t-values of interest ('sig' or 'not sig'
    - contrast_type: string that will go in name of the plot to save (e.g., "sex")
    - save_plots: True/False if want to save screenshots of plotted hemispheres and network breakdown of significant differences
    - output_dir: path to output directory (e.g. resdir_fig, i.e. '/data/p_02667/sex_diff_gradients/results/figures/')
    
    '''
    
    
    ### Arrange data (t-vals) in triangle format
    
    triangle = []
    
    triangle.append([1]+BN_dispersion_contrast_res.iloc[:6,df_column_index_with_tvals].tolist())
    triangle.append(2*[1]+BN_dispersion_contrast_res.iloc[6:11,df_column_index_with_tvals].tolist())
    triangle.append(3*[1]+BN_dispersion_contrast_res.iloc[11:15,df_column_index_with_tvals].tolist())
    triangle.append(4*[1]+BN_dispersion_contrast_res.iloc[15:18,df_column_index_with_tvals].tolist())
    triangle.append(5*[1]+BN_dispersion_contrast_res.iloc[18:20,df_column_index_with_tvals].tolist())
    triangle.append(6*[1]+BN_dispersion_contrast_res.iloc[20:21,df_column_index_with_tvals].tolist())
    triangle.append(7*[1])
    
    triangle = np.array(triangle)
    
    
    # do the same for the significance determination)
    triangle_sig = []
    
    triangle_sig.append([1]+BN_dispersion_contrast_res.iloc[:6,df_column_index_with_sigs].tolist())
    triangle_sig.append(2*[1]+BN_dispersion_contrast_res.iloc[6:11,df_column_index_with_sigs].tolist())
    triangle_sig.append(3*[1]+BN_dispersion_contrast_res.iloc[11:15,df_column_index_with_sigs].tolist())
    triangle_sig.append(4*[1]+BN_dispersion_contrast_res.iloc[15:18,df_column_index_with_sigs].tolist())
    triangle_sig.append(5*[1]+BN_dispersion_contrast_res.iloc[18:20,df_column_index_with_sigs].tolist())
    triangle_sig.append(6*[1]+BN_dispersion_contrast_res.iloc[20:21,df_column_index_with_sigs].tolist())
    triangle_sig.append(7*[1])
    
    triangle_sig = np.array(triangle_sig)
    
    
    
    
    # Create a mask for the upper triangle
    mask = np.triu(np.ones(triangle.shape, dtype=bool), k=1)
    
    # Set values in the lower triangle to NaN or 0
    triangle[np.logical_not(mask)] = np.nan  # Use np.nan to leave the lower triangle empty
    
    
    ### Plot
    
    plt.figure(figsize=(20, 18))
    
    plt.imshow(triangle, cmap='PuOr', interpolation='none', vmin=colormap_vmin, vmax=colormap_vmax, alpha=0.6)

    
    colorbar = plt.colorbar()
    colorbar.ax.tick_params(labelsize=40)  # Set the font size for the color bar ticks
    colorbar.set_label('t-values', fontsize=40)  # Set the font size for the color bar label
    #plt.title(‘t-vales sex effects on pairwise BN dispersion’, fontsize=30)
    plt.xticks(ticks=range(triangle.shape[0]), labels=np.array(['V', 'SM', 'DA', 'VA', 'L', 'FP', 'DMN']), rotation=0,  fontsize=40)
    plt.yticks(ticks=range(triangle.shape[0]), labels=np.array(['V', 'SM', 'DA', 'VA', 'L', 'FP', 'DMN']), rotation=0,  fontsize=40)
    
    # Add labels to the values in the upper triangle
    for i in range(triangle.shape[0]):
        for j in range(i + 1, triangle.shape[1]):
            value = triangle[i, j]
            if not np.isnan(value):
                annotation = f'{value:.2f}'
                
                plt.annotate(annotation, xy=(j, i), color='black', ha='center', va='center', fontsize=40)
                
                
                if triangle_sig[i, j] == 'sig':
                    #annotation += '*'
                    plt.annotate('*', xy=(j, i - 0.25), color='red', ha='center', va='center', fontsize=40)
    
                #plt.annotate(annotation, xy=(j, i), color='black', ha='center', va='center', fontsize=40)
    
    # Save the plot 
    if save_plot:
        
        plt.savefig(output_dir+'BN_dispersion_heatmap_matrix_tvals_'+contrast_type+'_contrast.png', dpi=300,  bbox_inches='tight')
    
    plt.show()
    

In [6]:
def print_plot_corr_networks(x, y, x_label, y_label, seven_networks=True, corr_type = 'spearman'):
    
    '''
    
    
    Output:
    - correlations between 2 variables, both overall and per network
    - scatterplots colorcoded by yeo network, with regression lines per network
    - seven_networks=True: indicate if using 7 or 17 Yeo network solution
    - corr_type: for output correlation breakdown, indicate person or spearman
    
    
    '''
    
    ### creating a dataframe in order to have the data in the correct format to be plotted
    if seven_networks:
        temp_dict = {x_label: x, y_label: y, 'yeo_network': yeo7_networks_array_labels}  
        
    else: 
        temp_dict = {x_label: x, y_label: y, 'yeo_network': yeo_17_2_7_networks_array_labels} 
    dataframe = pd.DataFrame(data = temp_dict)
    

    ### print overall correlation
    print('Note that the correlation p-values below have not undergone permutation testing! so here the correlations (r coefficients) are only indicative of effect sizes\n\n')
    
    print(f"Overall Pearson correlation between {x_label} and {y_label}: r = {round(stats.pearsonr(dataframe[x_label], dataframe[y_label])[0], 2)}; p = {round(stats.pearsonr(dataframe[x_label], dataframe[y_label])[1], 3)}\n")
    
    network_labels = ['visual', 'sensory motor', 'dorsal attention', 'ventral attention', 'limbic', 'fronto parietal', 'DMN']

    for i in range(len(network_labels)):
        
        if corr_type == 'spearman':
            corr_coef = stats.spearmanr(dataframe.loc[dataframe['yeo_network'] == network_labels[i]][x_label], dataframe.loc[dataframe['yeo_network'] == network_labels[i]][y_label])[0]
            p_val = stats.spearmanr(dataframe.loc[dataframe['yeo_network'] == network_labels[i]][x_label], dataframe.loc[dataframe['yeo_network'] == network_labels[i]][y_label])[1]
            
            print(f"{network_labels[i]}: r = {round(corr_coef, 2)}, p = {round(p_val, 3)}")
            
            
        elif corr_type == 'pearson':
            corr_coef = stats.pearsonr(dataframe.loc[dataframe['yeo_network'] == network_labels[i]][x_label], dataframe.loc[dataframe['yeo_network'] == network_labels[i]][y_label])[0]
            p_val = stats.pearsonr(dataframe.loc[dataframe['yeo_network'] == network_labels[i]][x_label], dataframe.loc[dataframe['yeo_network'] == network_labels[i]][y_label])[1]
        
            print(f"{network_labels[i]}: r = {round(corr_coef, 2)}, p = {round(p_val, 3)}")
    
    print("\n")
    
    
    ### scatter plot color-coded by network, with regression lines 
        
    # original Yeo network colors
    palette_labeled_networks = {'DMN': 'indianred',  
                                'dorsal attention' : 'forestgreen',  
                                'fronto parietal' : 'orange',  
                                'limbic' : 'lemonchiffon',  
                                'sensory motor' : 'steelblue',
                                'ventral attention' : 'orchid', 
                                'visual' : 'darkorchid'} 

    # plot
    
    sns.lmplot(x = x_label, y = y_label, 
           hue = 'yeo_network',
           data = dataframe,
           palette = palette_labeled_networks, 
           height=10, aspect=1.2,  # aspect gives you the height-width ratio
           legend=False)

    sns.despine()

    plt.title(f"Correlation between {x_label} and {y_label}", y=1.05, fontsize=25)
    plt.xlabel(x_label, fontsize=25)
    plt.ylabel(y_label, fontsize=25)
    plt.tick_params(labelsize=25)
    plt.legend(fontsize=25, bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)